# Impute downsampled

In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import anndata
import matplotlib.pyplot as plt

In [2]:
root = Path("/tscc/projects/ps-epigen/users/biy022/scmethylhic/human_hippocampus/snm3c/")
hic_path = root / "Combined" / "hic"
downsample_dir = hic_path / "downsampled"

In [3]:
impute_resolutions = ["10k", "25k", "100k", "250k", "500k", "1M", "6M"]
impute_resolution_numeric = {
    "10k": 10000,
    "25k": 25000,
    "100k": 100000,
    "250k": 250000,
    "500k": 500000,
    "1M": 1000000,
    "6M": 6000000
}
age_group = ["G20-40", "G40-60", "G60-80", "G80-100"]

In [5]:
for dir in downsample_dir.glob("*"):
    for res in impute_resolutions:
        for age in age_group:
            (dir / "impute" / res / age).mkdir(parents=True, exist_ok=True)

In [7]:
sbatch_header = (
    "#! /bin/bash\n"
    "#SBATCH -p condo\n"
    "#SBATCH -q condo\n"
    "#SBATCH -A csd772\n"
    "#SBATCH -J {0}_{1}_{2}\n"
    "#SBATCH -N 2\n"
    "#SBATCH -c 12\n"
    "#SBATCH --mem 200G\n"
    "#SBATCH -t 8:00:00\n"
    "#SBATCH -o /tscc/projects/ps-epigen/users/biy022/scmethylhic/human_hippocampus/snm3c/Combined/hic/downsampled/{0}/impute/{1}/{2}/sbatch.out\n"
    "#SBATCH -e /tscc/projects/ps-epigen/users/biy022/scmethylhic/human_hippocampus/snm3c/Combined/hic/downsampled/{0}/impute/{1}/{2}/sbatch.err\n"
    "#SBATCH --mail-user biy022@health.ucsd.edu\n"
    "#SBATCH --mail-type FAIL\n"
    "\n"
    "source ~/.bashrc\n"
    "conda activate schicluster\n"
    "bash {3}\n"
)

In [7]:
for file in downsample_dir.glob("Micro/impute/10k/G*/snakemake_cmd.txt"):
    age_group = file.parts[-2]
    resolution = file.parts[-3]
    subclass = file.parts[-5]

    sbatch_string = sbatch_header.format(
        subclass,
        resolution,
        age_group,
        str(file)
    )

    with open(downsample_dir / subclass / "impute" / resolution / age_group / "command.sbatch", "w") as f:
        f.write(sbatch_string)

In [8]:
list_of_celltypes = [
    "Astro", "CA", "DG", "Endo_VLMC", "NR2F2-LAMP5", 
    "Oligo", "OPC", "PVALB", "SST", "SUB", "VIP"
]
for celltype in list_of_celltypes:
    for file in downsample_dir.glob("{}/impute/10k/G*/snakemake_cmd.txt".format(celltype)):
        age_group = file.parts[-2]
        resolution = file.parts[-3]
        subclass = file.parts[-5]
    
        sbatch_string = sbatch_header.format(
            subclass,
            resolution,
            age_group,
            str(file)
        )
    
        with open(downsample_dir / subclass / "impute" / resolution / age_group / "command.sbatch", "w") as f:
            f.write(sbatch_string)